# Review

We've build an agent with memory:

- `act` - tool node executes a tool
- `observe` - pass the tool node outputs back to the LLM
- `reason` - LLM  reasons about to do next (e.g., call another tool)
- `persist state` - using in memory checkpointer for long-running conversations

# Goals

Now we'll cover how to actually deploy your agent!

We've cover deployment locally and to `LangGraph Cloud`

# Concepts

There are a few concepts to understand.

`LangGraph`

- Python and Javascript library
- Allows creation of agentic workflows

`LangGraph  API`

- Blundles the graph code
- Provides a task queue for managing asynchronous operations
- Offers persistence for maintaining state across interactions

`LangGraph Cloud`

- Hosted service for the LangGraph API
- Allows deployment of graphs from GitHub repositories
- Provides monitoring, tracing, and API documentation for deployed graphs
- Accessible via a unique URL for each deployment

`LangGraph Studio`

- Integrated Development Environment (IDE) for LangGraph applications
- Uses the API as its back-end, allowing reaal-time testing and exploration of graphs
- Can be run locally or with cloud-deployment

`LangGraph SDK`

- Python library for programmatically interacting with LangGraph graphs
- Provides a consistent interface for working with graphs, wheter served locally or in the cloud.
- Allows creation of clients, access to assistants, thread management, and execution of runs.

## Testing Locally

We can easily connect with graphs that are served locally in LangGraph Studio!

We do this via the `url` provided in API field in the image below.

<center><img src="../../../assets/langgraph-cli.png" width=900 alt="langgraph-cli"></img></center>

In [1]:
from langgraph_sdk import get_client

In [3]:
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

In [4]:
# Search all hosted graphs
assistants = await client.assistants.search()
assistants

[{'assistant_id': '166c0d22-706c-5a5b-9027-ea37f7308a85',
  'graph_id': 'react',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'react',
  'created_at': '2024-12-29T16:26:28.065150+00:00',
  'updated_at': '2024-12-29T16:26:28.065150+00:00',
  'version': 1},
 {'assistant_id': '228f9934-0cdd-5383-92c8-ee8422522cc2',
  'graph_id': 'router',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'router',
  'created_at': '2024-12-29T16:26:27.689297+00:00',
  'updated_at': '2024-12-29T16:26:27.689297+00:00',
  'version': 1},
 {'assistant_id': '608f7b9c-04e5-5cf2-a3f7-ba75a3719808',
  'graph_id': 'chain',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'chain',
  'created_at': '2024-12-29T16:26:27.316657+00:00',
  'updated_at': '2024-12-29T16:26:27.316657+00:00',
  'version': 1},
 {'assistant_id': '5e0f6b46-d00f-5af4-87dc-ea2105ff526d',
  'graph_id': 'simple',
  'config': {},
  'metadata': {'created_by': 'system'},
  'name': 'simple',
  'create

In [5]:
# Sellectthe ReAct Agent
assistant = assistants[0]
assistant

{'assistant_id': '166c0d22-706c-5a5b-9027-ea37f7308a85',
 'graph_id': 'react',
 'config': {},
 'metadata': {'created_by': 'system'},
 'name': 'react',
 'created_at': '2024-12-29T16:26:28.065150+00:00',
 'updated_at': '2024-12-29T16:26:28.065150+00:00',
 'version': 1}

In [6]:
# Let's create a thread to manage and track the conversation
thread = await client.threads.create()

In [7]:
from langchain_core.messages import HumanMessage

# Input
message = {"messages": [HumanMessage(content="Multiply 2 by 3")]}

# Stream
async for chunk in client.runs.stream(
    thread["thread_id"], assistant["assistant_id"], input=message, stream_mode="values"
):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data["messages"][-1])

{'content': 'Multiply 2 by 3', 'additional_kwargs': {'additional_kwargs': {}, 'response_metadata': {}, 'example': False}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'de063df0-8368-4a9e-bd43-b03eb12de6db', 'example': False}
{'content': '', 'additional_kwargs': {}, 'response_metadata': {'model': 'llama3.1-tool', 'created_at': '2025-01-02T14:13:46.0557478Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6241571300, 'load_duration': 4814522700, 'prompt_eval_count': 375, 'prompt_eval_duration': 275000000, 'eval_count': 22, 'eval_duration': 635000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, 'type': 'ai', 'name': None, 'id': 'run-80fff94e-aaa4-4bbf-895d-1910e1a8e353-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': '8d1667d3-bfea-44aa-87ba-a7508e384640', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 375, 'output_tokens': 22, 'total_tokens': 39

# Testing with the Cloud

We can deploy to Cloud via LangSmith